In [1]:
import pandas as pd

data_path = 'Data.csv'
df = pd.read_csv(data_path)

df.head()

,Country Name,Country Code,Series Name,Series Code,2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016]
0,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,13.82631955,3.924983822,21.39052839,14.36244147,0.426354785,12.75228709,5.600744658,2.724543364,1.45131466,2.260314201
1,Afghanistan,AFG,"Foreign direct investment, net outflows (% of ...",BM.KLT.DINV.WD.GD.ZS,..,-0.018972992,0.001990238,-0.007870805,0.006386583,-0.044510496,0.00266707,-9.34E-05,0.011304631,0.077310687
2,Afghanistan,AFG,Exports of goods and services (% of GDP),NE.EXP.GNFS.ZS,..,..,..,..,..,..,..,..,..,..
3,Afghanistan,AFG,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,..,1340,1570,1770,1750,1960,2070,2120,2140,2050
4,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,59.111,59.852,60.364,60.851,61.419,61.923,62.417,62.545,62.659,63.136


In [2]:
df = df.drop(["Series Code","Country Code"], axis = 1)

In [3]:
df.replace("..", pd.NA, inplace=True)

year_columns = [col for col in df.columns if "YR" in col]

# Melt the dataframe with the correct year columns
melted_data = df.melt(id_vars=["Country Name", "Series Name"], value_vars=year_columns, var_name="Year", value_name="Value")

# Extract year from the 'Year' column (format: 20XX [YR20XX])
melted_data['Year'] = melted_data['Year'].str.extract('(\d+)').astype(int)

# Pivot the table to get the desired structure
restructured_data = melted_data.pivot_table(index=["Country Name", "Year"], columns="Series Name", values="Value").reset_index()

In [4]:
missing_values = restructured_data.isnull().sum()

In [5]:
missing_values

Series Name
Country Name                                                                   0
Year                                                                           0
Adjusted savings: net national savings (current US$)                         544
Exports of goods and services (% of GDP)                                     346
Final consumption expenditure (% of GDP)                                     408
Foreign direct investment, net outflows (% of GDP)                           517
GDP growth (annual %)                                                        101
GNI per capita, PPP (current international $)                                248
Government expenditure on education, total (% of government expenditure)     797
Gross domestic savings (% of GDP)                                            408
Imports of goods and services (% of GDP)                                     346
Land area (sq. km)                                                            27
Life expectancy 

In [6]:
restructured_data.drop(columns = ['Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)'], inplace = True)

In [7]:
cleaned_data = restructured_data.dropna()
cleaned_data

Series Name,Country Name,Year,Adjusted savings: net national savings (current US$),Exports of goods and services (% of GDP),Final consumption expenditure (% of GDP),"Foreign direct investment, net outflows (% of GDP)",GDP growth (annual %),"GNI per capita, PPP (current international $)","Government expenditure on education, total (% of government expenditure)",Gross domestic savings (% of GDP),Imports of goods and services (% of GDP),Land area (sq. km),"Life expectancy at birth, total (years)","Population, total",Tax revenue (% of GDP),"Unemployment, total (% of total labor force) (modeled ILO estimate)"
14,Albania,2011,8.734919e+08,29.241537,89.273599,1.567712,2.545406,10210.0,11.106843,10.726401,51.977032,27400.0,78.092,2905195.0,18.022013,13.480
15,Albania,2012,6.461165e+08,28.937488,88.811696,0.675159,1.417243,10450.0,11.626995,11.188304,47.572713,27400.0,78.064,2900401.0,17.483234,13.380
16,Albania,2013,4.381194e+08,28.916315,91.054867,0.222676,1.002018,10750.0,11.580032,8.945133,46.957399,27400.0,78.123,2895092.0,16.503271,15.870
17,Albania,2014,1.894569e+08,28.213001,92.239223,0.571868,1.774449,11360.0,10.988461,7.760777,47.194844,27400.0,78.407,2889104.0,18.301936,18.050
18,Albania,2015,-1.720922e+07,27.267391,91.805395,0.696510,2.218726,11800.0,10.441024,8.194605,44.533615,27400.0,78.644,2880703.0,18.515790,17.190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,Zambia,2016,3.034550e+09,35.324676,67.483271,0.843193,3.776679,3220.0,15.663620,32.516729,38.633886,743390.0,61.794,16767761.0,13.353181,8.431
2163,Zimbabwe,2010,-1.235104e+09,29.640895,105.079099,0.359585,21.452061,1660.0,8.536720,-5.079099,53.483295,386850.0,50.652,12839771.0,17.434924,5.542
2164,Zimbabwe,2011,-1.309389e+09,34.800802,102.467158,0.302796,14.620207,1850.0,24.097687,-2.467158,54.665725,386850.0,53.347,13025785.0,17.928254,5.370
2165,Zimbabwe,2012,-3.632235e+09,25.163254,113.979050,0.286301,15.744877,2090.0,29.751089,-13.979050,48.999281,386850.0,55.626,13265331.0,17.835489,5.020


In [8]:
cleaned_data.to_csv("data_raw_2.csv")

In [9]:
def normalize_column(column):
    x_min = column.min()
    x_max = column.max()
    normalized_column = (column - x_min) / (x_max - x_min)
    return normalized_column


columns_to_normalize = cleaned_data.columns[2:]

cleaned_data.loc[:, columns_to_normalize] = cleaned_data.loc[:, columns_to_normalize].apply(normalize_column)


C:\Users\potat\AppData\Local\Temp\ipykernel_87752\3122651987.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data.loc[:, columns_to_normalize] = cleaned_data.loc[:, columns_to_normalize].apply(normalize_column)


In [10]:
cleaned_data

Series Name,Country Name,Year,Adjusted savings: net national savings (current US$),Exports of goods and services (% of GDP),Final consumption expenditure (% of GDP),"Foreign direct investment, net outflows (% of GDP)",GDP growth (annual %),"GNI per capita, PPP (current international $)","Government expenditure on education, total (% of government expenditure)",Gross domestic savings (% of GDP),Imports of goods and services (% of GDP),Land area (sq. km),"Life expectancy at birth, total (years)","Population, total",Tax revenue (% of GDP),"Unemployment, total (% of total labor force) (modeled ILO estimate)"
14,Albania,2011,0.127541,0.115956,0.425770,0.228571,0.515895,0.071837,0.282014,0.574230,0.204508,0.001671,0.817177,0.001948,0.096265,0.482143
15,Albania,2012,0.127463,0.114611,0.422660,0.226273,0.491705,0.073770,0.298476,0.577340,0.182100,0.001671,0.816344,0.001945,0.092508,0.478499
16,Albania,2013,0.127392,0.114517,0.437764,0.225108,0.482802,0.076186,0.296990,0.562236,0.178969,0.001671,0.818101,0.001941,0.085676,0.569242
17,Albania,2014,0.127306,0.111404,0.445739,0.226007,0.499364,0.081098,0.278267,0.554261,0.180177,0.001671,0.826558,0.001937,0.098217,0.648688
18,Albania,2015,0.127235,0.107219,0.442818,0.226328,0.508891,0.084642,0.260942,0.557182,0.166638,0.001671,0.833616,0.001931,0.099708,0.617347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,Zambia,2016,0.128284,0.142878,0.279046,0.226706,0.542297,0.015543,0.426229,0.720954,0.136622,0.045391,0.331815,0.011939,0.063713,0.298141
2163,Zimbabwe,2010,0.126817,0.117724,0.532196,0.225461,0.921296,0.002980,0.200674,0.467804,0.212171,0.023620,0.000000,0.009108,0.092172,0.192857
2164,Zimbabwe,2011,0.126791,0.140560,0.514609,0.225315,0.774806,0.004510,0.693154,0.485391,0.218187,0.023620,0.080258,0.009242,0.095611,0.186589
2165,Zimbabwe,2012,0.125993,0.097907,0.592124,0.225272,0.798921,0.006443,0.872075,0.407876,0.189358,0.023620,0.148128,0.009415,0.094964,0.173834


In [11]:
column_mapping = {
    'Exports of goods and services (% of GDP)': 'Exports',
    'Foreign direct investment, net outflows (% of GDP)': 'FDI outflow',
    'GDP growth (annual %)': 'GDP growth',
    'GNI per capita, PPP (current international $)': 'Income Measure',
    'Government expenditure on education, total (% of government expenditure)': 'Education expenditure',
    'Imports of goods and services (% of GDP)': 'Imports',
    'Land area (sq. km)': 'Geographic Size',
    'Life expectancy at birth, total (years)': 'Life expectancy',
    'Population, total': 'Population',
    'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)': 'Poverty headcount',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)': 'Unemployment',
    'Adjusted savings: net national savings (current US$)': "Resident's Savings",
    'Final consumption expenditure (% of GDP)': "FCE",
    'Gross domestic savings (% of GDP)': "GDS",
    "Tax revenue (% of GDP)": "Tax revenue"
}

# Rename columns using the mapping dictionary
cleaned_data.rename(columns=column_mapping, inplace=True)


C:\Users\potat\AppData\Local\Temp\ipykernel_87752\1005343960.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data.rename(columns=column_mapping, inplace=True)


In [12]:
cleaned_data

Series Name,Country Name,Year,Resident's Savings,Exports,FCE,FDI outflow,GDP growth,Income Measure,Education expenditure,GDS,Imports,Geographic Size,Life expectancy,Population,Tax revenue,Unemployment
14,Albania,2011,0.127541,0.115956,0.425770,0.228571,0.515895,0.071837,0.282014,0.574230,0.204508,0.001671,0.817177,0.001948,0.096265,0.482143
15,Albania,2012,0.127463,0.114611,0.422660,0.226273,0.491705,0.073770,0.298476,0.577340,0.182100,0.001671,0.816344,0.001945,0.092508,0.478499
16,Albania,2013,0.127392,0.114517,0.437764,0.225108,0.482802,0.076186,0.296990,0.562236,0.178969,0.001671,0.818101,0.001941,0.085676,0.569242
17,Albania,2014,0.127306,0.111404,0.445739,0.226007,0.499364,0.081098,0.278267,0.554261,0.180177,0.001671,0.826558,0.001937,0.098217,0.648688
18,Albania,2015,0.127235,0.107219,0.442818,0.226328,0.508891,0.084642,0.260942,0.557182,0.166638,0.001671,0.833616,0.001931,0.099708,0.617347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,Zambia,2016,0.128284,0.142878,0.279046,0.226706,0.542297,0.015543,0.426229,0.720954,0.136622,0.045391,0.331815,0.011939,0.063713,0.298141
2163,Zimbabwe,2010,0.126817,0.117724,0.532196,0.225461,0.921296,0.002980,0.200674,0.467804,0.212171,0.023620,0.000000,0.009108,0.092172,0.192857
2164,Zimbabwe,2011,0.126791,0.140560,0.514609,0.225315,0.774806,0.004510,0.693154,0.485391,0.218187,0.023620,0.080258,0.009242,0.095611,0.186589
2165,Zimbabwe,2012,0.125993,0.097907,0.592124,0.225272,0.798921,0.006443,0.872075,0.407876,0.189358,0.023620,0.148128,0.009415,0.094964,0.173834


In [13]:
region_definitions = {
'Africa': [
        'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon', 'Congo, Rep.', "Cote d'Ivoire", 'Egypt, Arab Rep.',
        'Eswatini', 'Gambia, The', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Madagascar', 'Malawi', 'Mali', 'Mauritania',
        'Mauritius', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Senegal', 'Sierra Leone', 'South Africa', 'Sudan', 'Tanzania',
        'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe'
    ],
    'Asia': [
        'Armenia', 'Azerbaijan', 'Bangladesh', 'Bhutan', 'Brunei Darussalam', 'Cambodia', 'China', 'Georgia', 'India', 'Indonesia',
        'Iran, Islamic Rep.', 'Iraq', 'Israel', 'Japan', 'Jordan', 'Kazakhstan', 'Korea, Rep.', 'Kuwait', 'Kyrgyz Republic', 'Lao PDR',
        'Lebanon', 'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Nepal', 'Oman', 'Pakistan', 'Philippines', 'Qatar', 'Saudi Arabia',
        'Singapore', 'Sri Lanka', 'Syrian Arab Republic', 'Tajikistan', 'Thailand', 'Timor-Leste', 'Turkey', 'Turkmenistan', 'United Arab Emirates',
        'Uzbekistan', 'Viet Nam', 'Yemen'
    ],
    'Europe': [
        'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic',
        'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Lithuania',
        'Luxembourg', 'Malta', 'Moldova', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russian Federation', 'Slovak Republic',
        'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Ukraine', 'United Kingdom'
    ],
    'North America': [
        'Belize', 'Canada', 'Costa Rica', 'Dominican Republic', 'El Salvador', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico',
        'Nicaragua', 'Panama', 'Trinidad and Tobago', 'United States'
    ],
    'South America': [
        'Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'
    ],
    'Oceania': [
        'Australia', 'Fiji', 'New Zealand', 'Papua New Guinea', 'Samoa', 'Solomon Islands', 'Vanuatu'
]}

# Function to map each country to its region
def get_country_region(country):
    for region, countries in region_definitions.items():
        if country in countries:
            return region
    return 'Undefined'  # For countries not listed in the region_definitions

# Add a 'Region' column to the DataFrame
cleaned_data['Region'] = cleaned_data['Country Name'].apply(get_country_region)

C:\Users\potat\AppData\Local\Temp\ipykernel_87752\1561263818.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['Region'] = cleaned_data['Country Name'].apply(get_country_region)


In [14]:
cleaned_data

Series Name,Country Name,Year,Resident's Savings,Exports,FCE,FDI outflow,GDP growth,Income Measure,Education expenditure,GDS,Imports,Geographic Size,Life expectancy,Population,Tax revenue,Unemployment,Region
14,Albania,2011,0.127541,0.115956,0.425770,0.228571,0.515895,0.071837,0.282014,0.574230,0.204508,0.001671,0.817177,0.001948,0.096265,0.482143,Europe
15,Albania,2012,0.127463,0.114611,0.422660,0.226273,0.491705,0.073770,0.298476,0.577340,0.182100,0.001671,0.816344,0.001945,0.092508,0.478499,Europe
16,Albania,2013,0.127392,0.114517,0.437764,0.225108,0.482802,0.076186,0.296990,0.562236,0.178969,0.001671,0.818101,0.001941,0.085676,0.569242,Europe
17,Albania,2014,0.127306,0.111404,0.445739,0.226007,0.499364,0.081098,0.278267,0.554261,0.180177,0.001671,0.826558,0.001937,0.098217,0.648688,Europe
18,Albania,2015,0.127235,0.107219,0.442818,0.226328,0.508891,0.084642,0.260942,0.557182,0.166638,0.001671,0.833616,0.001931,0.099708,0.617347,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,Zambia,2016,0.128284,0.142878,0.279046,0.226706,0.542297,0.015543,0.426229,0.720954,0.136622,0.045391,0.331815,0.011939,0.063713,0.298141,Africa
2163,Zimbabwe,2010,0.126817,0.117724,0.532196,0.225461,0.921296,0.002980,0.200674,0.467804,0.212171,0.023620,0.000000,0.009108,0.092172,0.192857,Africa
2164,Zimbabwe,2011,0.126791,0.140560,0.514609,0.225315,0.774806,0.004510,0.693154,0.485391,0.218187,0.023620,0.080258,0.009242,0.095611,0.186589,Africa
2165,Zimbabwe,2012,0.125993,0.097907,0.592124,0.225272,0.798921,0.006443,0.872075,0.407876,0.189358,0.023620,0.148128,0.009415,0.094964,0.173834,Africa


In [15]:
cleaned_data.to_csv("Normalized_data_2.csv")